In [1]:
from torch.utils.data import DataLoader
from torch import cuda as torch_cuda, device as torch_device, backends as torch_backends
import data_loader
import bpe
import gpt1

In [2]:
# если есть граф процессор от Nvidia, то на нем. Если нет, то на cpu

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = "cpu"
if torch_cuda.is_available():
    device = torch_device('cuda:0')  
elif torch_backends.mps.is_available():
    device = torch_device("mps")
else: 
    device = torch_device('cpu')

device

device(type='mps')

# Train Validate Loaders

In [3]:
data_file = '../data/pushkin-onegin.txt'
bpe_file = './models/bpe_onegin.dill'
seq_len=100
batch_size=50
# device='cpu'

# доля обучающей выборки
train_len = 0.8

In [4]:
text = ''
with open(data_file) as f:
    text=f.read()

text[:100],len(text)

('Роман в стихах\nНе мысля гордый свет забавить,\nВниманье дружбы возлюбя,\nХотел бы я тебе представить\nЗ',
 142592)

In [5]:
tokenizer = bpe.BPE.load(bpe_file)
len(tokenizer.id2token)

Объект загружен из ./models/bpe_onegin.dill


2000

In [6]:
tokens = tokenizer.encode(text)
len(tokens)

50412

In [7]:
# разделил на выборку на train и valid
n = int(train_len*len(tokens))
train_tokens = tokens[:n]
valid_tokens = tokens[n:]

In [8]:
# данные для train
train_data = data_loader.GetData(data= train_tokens, seq_len= seq_len, device= device)
# данные для валидации
valid_data = data_loader.GetData(data= valid_tokens, seq_len= seq_len, device= device)

In [9]:
# загрузчик для train
train_loader = DataLoader(dataset=train_data,batch_size=batch_size)
# загрузчик для валидации
valid_loader = DataLoader(dataset=valid_data,batch_size=batch_size)

# GPT.fit

In [10]:
vocab_size = tokenizer.vocab_size
max_seq_len = 300
emb_size = 500
num_heads=6
head_size=64
num_layers=6
dropout=0.1
num_epoch=5
learning_rate=2.5e-4

In [11]:
gpt = gpt1.GPT(
    vocab_size= vocab_size,
    max_seq_len=max_seq_len,
    emb_size= emb_size,
    num_heads=1,
    head_size=1,
    num_layers=1,
    dropout=0.1,
    device=device
)

In [12]:
gpt.fit(
    train_loader=train_loader,
    valid_loader=valid_loader,
    num_epoch=num_epoch,
    learning_rate=learning_rate,
    prn=True
)

Epoch [1/5], Loss on train: 7.0042
Epoch [1/5], Loss on validation: 1.6677
Epoch [2/5], Loss on train: 5.7649
Epoch [2/5], Loss on validation: 1.5868
Epoch [3/5], Loss on train: 4.9173
Epoch [3/5], Loss on validation: 1.5975
Epoch [4/5], Loss on train: 4.4274
Epoch [4/5], Loss on validation: 1.6352
Epoch [5/5], Loss on train: 4.1305
Epoch [5/5], Loss on validation: 1.6795
